In [1]:
#import libraries

import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, callback



In [2]:
#loading the gapminder data
df = pd.read_csv("gdp_pcap.csv")

#checking data 
print(df.head())

melted_df = pd.melt(df, id_vars=['country'], var_name='year', value_name='gdp')

#check and displays df that has all the countries within a year --> year has separate col
melted_df.head(70)


       country  1800  1801  1802  1803  1804  1805  1806  1807  1808  ...  \
0  Afghanistan   599   599   599   599   599   599   599   599   599  ...   
1       Angola   465   466   469   471   472   475   477   479   481  ...   
2      Albania   585   587   588   590   592   593   595   597   598  ...   
3      Andorra  1710  1710  1710  1720  1720  1720  1730  1730  1730  ...   
4          UAE  1420  1430  1430  1440  1450  1450  1460  1460  1470  ...   

    2091   2092   2093   2094   2095   2096   2097   2098   2099   2100  
0   4800   4910   5030   5150   5270   5390   5520   5650   5780   5920  
1  24.8k  25.3k  25.9k  26.4k  26.9k  27.4k    28k  28.5k  29.1k  29.6k  
2    54k  54.6k  55.2k  55.8k  56.4k  56.9k  57.5k  58.1k  58.7k  59.2k  
3  79.3k  79.5k  79.8k  80.1k  80.4k  80.7k    81k  81.2k  81.5k  81.8k  
4  92.5k  92.6k  92.6k  92.7k  92.8k  92.9k  92.9k    93k  93.1k  93.1k  

[5 rows x 302 columns]


,country,year,gdp
0,Afghanistan,1800,599
1,Angola,1800,465
2,Albania,1800,585
3,Andorra,1800,1710
4,UAE,1800,1420
...,...,...,...
65,Gambia,1800,991
66,Guinea-Bissau,1800,613
67,Equatorial Guinea,1800,454
68,Greece,1800,1620


In [3]:
min_year = int(melted_df["year"].min())
print(min_year)

max_year = int(melted_df["year"].max())
print(max_year)

print(type(max_year))


1800
2100
<class 'int'>


In [4]:
#getting rid of the Ks in the value column

def val_float(x):
    if type(x) == float or type(x)== int:
        return x #if it is a float or int, then keep as is
    if 'k' in x:
        if len(x) > 1 :
            return float(x.replace('k', '')) * 1000
        return 1000.0


melted_df['gdp'] = melted_df['gdp'].apply(val_float)




In [6]:
df = melted_df  # Assuming this contains the melted data

#import style sheets 
externalstylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] # load the CSS stylesheet

app = Dash(__name__, external_stylesheets = externalstylesheets)

server = app.server

# Define layout
app.layout = html.Div([

    # Title and description
    html.Div([
        html.H1('App for GDP in Countries from Year 1800 - 2100'),
        html.P('Description: This is a dashboard that displays the GDP per capita of each country from the year 1800 to 2100. '
               'The data tracks 195 countries throughout the years and their respective GDPs.'
               'There is a dropdown and slider bar to select the country and years that you are interested in investigating.'
               'The data does include years past the current date, which are meant to serve as estimations of future GDP.')
    ]),

    # Dropdown for countries
    html.Div([
            dcc.Dropdown(
                options=[{'label': country, 'value': country} for country in df['country'].unique()],
                value= ['Afghanistan', 'Angola', 'Albania'],
                id='country-dropdown',
                multi=True
            ),
    ]),

    # Slider for years
    html.Div([
            dcc.RangeSlider(
                min = min_year, 
                max = max_year, 
                step = 1, 
                value = [1800,1950], #default, note array notation
                id = 'year-slider',
                marks = {year: str(year) for year in range(1800, 2101, 10)} 
                    #marks is a dict that represents the numerical values and the labels
                    #need to have a mark for everything from 1800 to 2100, thus make a for loop for each mark 
                ),
    ], style={'width': '90%', 'margin': 'auto'}),  # Center the slider

    # Graph
    dcc.Graph(id='country-graph')
])


@app.callback(
    Output('country-graph', 'figure'),
    [Input('country-dropdown', 'value'),
    Input('year-slider', 'value')]
)
def update_graph(selected_countries, selected_year):

    #initialize the selected df from choices
    selected_df = df

    #if countries selected then filter for those rows
    if selected_countries:
        selected_df = selected_df[selected_df['country'].isin(selected_countries)]
    #if year selected then filter --> need to convert to make year col all int so can compare against min&max
    if selected_year:
        min_year = selected_year[0]
        max_year = selected_year[1]
        selected_df = selected_df[(selected_df['year'].astype(int) >= min_year) & (selected_df['year'].astype(int) <= max_year)]

    fig = px.line(selected_df, x='year', y='gdp', color='country')

#update the layout
    fig.update_layout(
        xaxis_title='Year',
        yaxis_title='GDP per capita',
        legend_title='Country'
    )

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)